In [1]:
import os
import numpy as np
import torch
from PIL import Image

In [ ]:
class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # 모든 이미지 파일들을 읽고, 정렬
        # 이미지와 분할 마스크 정렬을 확인
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))
        
    def __getitem__(self, idx):
        # 이미지와 마스크를 로딩
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # 분할 마스크는 RGB로 변환하지 않음
        # 왜냐하면 각 색상은 다른 인스턴스에 해당, 0은 배경에 해당
        mask = Image.open(mask_path)
        print('org :', mask[:1])
        
        # numpy 배열을 PIL 이미지로 변환
        mask = np.array(mask)
        print('change :', mask[:1])
        # 인스턴스들은 다른 색들로 인코딩 되어 있습니다.
        obj_ids = np.unique(mask)
        print('unique :', mask[:1])
        # 첫번째 id 는 배경이라 제거합니다
        obj_ids = obj_ids[1:]
        
        # 컬러 인코딩된 마스크를 바이너리 마스크 세트로 나눕니다
        masks = mask == obj_ids[:, None, None]
        
        # 각 마스크의 바운딩 박스 좌표를 얻습니다
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])